# Tutorial: Density estimation via Binless Multidimensional Integration

This tutorial showcases the performance of the BMTI method for density estimation.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from dadapy import DensityAdvanced
from scipy.ndimage.filters import gaussian_filter1d

%load_ext autoreload
%autoreload 2
%matplotlib inline

ImportError: cannot import name 'cython_clustering' from 'dadapy._cython' (/home/francesco/Desktop/dssc/robavaria/my_dadapy/DADApy/dadapy/_cython/__init__.py)

In [ ]:
# Load a 6 dimensional dataset from the dataset folder
X = np.genfromtxt("datasets/6d_double_well.txt")
true_log_den = np.genfromtxt("datasets/6d_double_well_logdensities_and_grads.txt")[:, 0]

# Subsample the dataset for a faster run
every = 1
X = X[2000::every]
true_log_den = true_log_den[2000::every]

print(X.shape)

In [ ]:
d = DensityAdvanced(X, maxk=100, verbose=True)

# copute the density using the kNN method
d.compute_density_kNN(k=10)
log_den_kNN = d.log_den

# Compute the density using the kstarNN method
d.compute_kstar()
d.compute_density_kstarNN()
log_den_kstarNN = d.log_den

# Compute the density using the BMTI method
d.compute_density_BMTI(solver = "sp_cg", delta_F_inv_cov = "uncorr", comp_log_den_err = "LSDI")
log_den_BMTI = d.log_den

In [ ]:
d.log_den_err

In [ ]:
# remove the mean to both the true and estimated density
true_log_den = true_log_den - np.mean(true_log_den)
log_den_kNN = log_den_kNN - np.mean(log_den_kNN)
log_den_kstarNN = log_den_kstarNN - np.mean(log_den_kstarNN)
log_den_BMTI = log_den_BMTI - np.mean(log_den_BMTI)

In [ ]:
# compute MSE errors
MSE_kNN = np.mean((log_den_kNN - true_log_den) ** 2)
MSE_kstarNN = np.mean((log_den_kstarNN - true_log_den) ** 2)
MSE_BMTI = np.mean((log_den_BMTI - true_log_den) ** 2)

print("MSE kNN: ", MSE_kNN)
print("MSE kstarNN: ", MSE_kstarNN)
print("MSE BMTI: ", MSE_BMTI)

In [ ]:
# plot real density vs estimated density
plt.figure(figsize=(5, 5))
plt.scatter(true_log_den, log_den_kNN, marker=".", label="kNN")
plt.scatter(true_log_den, log_den_kstarNN, marker=".", label="kstarNN")
plt.scatter(true_log_den, log_den_BMTI, marker=".", label="BMTI")
plt.plot(true_log_den, true_log_den, "k--")
plt.xlabel("True log density")
plt.ylabel("Estimated log density")
plt.legend()
plt.tight_layout()

In [ ]:
# compute Mean Absolute Errors (MAEs)
MAE_kNN = np.mean(np.abs(log_den_kNN - true_log_den))
MAE_kstarNN = np.mean(np.abs(log_den_kstarNN - true_log_den))
MAE_BMTI = np.mean(np.abs(log_den_BMTI - true_log_den))

print("MAE kNN: ", MAE_kNN)
print("MAE kstarNN: ", MAE_kstarNN)
print("MAE BMTI: ", MAE_BMTI)

In [ ]:
# sort indices in order of increasing true log-density
sortlden = true_log_den.argsort()

# plot MAE as a function of true log-density
plt.figure(figsize=(8, 5))

plt.plot(true_log_den[sortlden],gaussian_filter1d(np.abs((log_den_kNN-true_log_den)[sortlden]),sigma=200),label="kNN")
plt.plot(true_log_den[sortlden],gaussian_filter1d(np.abs((log_den_kstarNN-true_log_den)[sortlden]),sigma=200),label="kstarNN")
plt.plot(true_log_den[sortlden],gaussian_filter1d(np.abs((log_den_BMTI-true_log_den)[sortlden]),sigma=200),label="BMTI")


plt.xlabel("True log density")
plt.ylabel("Smoothed MAE")
plt.legend()
plt.tight_layout()